In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Thesis_Organized

/content/drive/MyDrive/Thesis_Organized


In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [4]:
import os
main_dir = os.getcwd()
main_dir

'/content/drive/MyDrive/Thesis_Organized'

In [5]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [7]:
Data_cfg = {
  # COCO TESTING DATASET PATH
  "Coco_labels_train_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_train_data.json",
  # COCO TRAINING VALIDATION PATH
  "Coco_labels_val_dir": "/content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json",
  # CROPPED IMAGES PATH
  "cropped_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Images_cropped",
  # ORIGINAL IMAGES PATH
  "original_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Original Images",
  # GROUND TRUTH IMAGES PATH
  "GT_Images_dir": "/content/drive/MyDrive/Thesis_Organized/Data/Ground Truth",
  # PATH TO THE CSV FILES OF THE ANNOTATIONS CREATED IN THE DATA PROCESSING PART
  "annotations_path": "Data/Ground Truth/annotations.csv"
}

In [8]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, Data_cfg["Coco_labels_train_dir"], os.path.join(Data_cfg["cropped_Images_dir"], "Train"))
register_coco_instances("my_dataset_val", {}, Data_cfg["Coco_labels_val_dir"], os.path.join(Data_cfg["cropped_Images_dir"], "Val"))

In [9]:
from detectron2.engine.hooks import HookBase
from detectron2.utils.logger import log_every_n_seconds
import detectron2.utils.comm as comm
import time
import datetime
import logging
import numpy as np

class LossEvalHook(HookBase):

    def __init__(self, eval_period, model, data_loader):

        self._model = model
        self._period = eval_period
        self._data_loader = data_loader

    def _do_loss_eval(self):

        total = len(self._data_loader)  # Total number of batches
        num_warmup = min(5, total - 1)  # Number of batches to skip for warm-up

        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []  # Store losses for each batch
        for idx, inputs in enumerate(self._data_loader):
            # Reset timing and loss calculation after warm-up period
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()  # Ensure synchronization in CUDA operations
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            # Log progress and ETA after warm-up period
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    f"Loss on Validation  done {idx + 1}/{total}. {seconds_per_img:.4f} s / img. ETA={eta}",
                    n=5,
                )
            loss_batch = self._get_loss(inputs)  # Compute loss for the current batch
            losses.append(loss_batch)
        mean_loss = np.mean(losses)  # Calculate mean loss
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()  # Synchronize across all processes

        return losses

    def _get_loss(self, data):

        metrics_dict = self._model(data)
        # Ensure all metrics are scalars and detach any tensors from the graph
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())  # Sum up the losses
        return total_losses_reduced

    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [10]:
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators


class CustomTrainer(DefaultTrainer):

    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1, LossEvalHook(
            20, # Frequency of calculation - every 20 iterations here
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg, True)
            )
        ))

        return hooks

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        coco_evaluator = COCOEvaluator(dataset_name, output_dir=output_folder)

        evaluator_list = [coco_evaluator]

        return DatasetEvaluators(evaluator_list)

In [ ]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [ ]:
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_fpn_backbone', 'FREEZE_AT': 0})

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/07 14:31:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R50_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/07 18:37:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/07 18:37:57 detectron2]: Loss on Validation  done 11/97. 0.0001 s / img. ETA=0:00:07
[04/07 18:38:03 detectron2]: Loss on Validation  done 53/97. 0.0001 s / img. ETA=0:00:05
[04/07 18:38:13 detectron2]: Loss on Validation  done 82/97. 0.0002 s / img. ETA=0:00:03
[04/07 18:38:15 d2.utils.events]:  eta: 0:07:14  iter: 5419  total_loss: 0.5978  loss_cls: 0.1269  loss_box_reg: 0.287  loss_mask: 0.1527  loss_rpn_cls: 0.008497  loss_rpn_loc: 0.02509  validation_loss: 0.7232    time: 0.8294  last_time: 0.7182  data_time: 0.8351  last_data_time: 0.4410   lr: 0.001  max_mem: 6493M
WARNING [04/07 18:38:33 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/07 18:38:33 d2.data.datasets.coco]: Loaded 97 images in COCO format from /content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json
[04/07 18:38:33 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_lengt

In [ ]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "smooth_l1"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_X101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/07 18:46:58 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

X-101-32x8d.pkl: 356MB [00:02, 172MB/s]                           


[04/07 18:47:00 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[04/07 18:47:01 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up - Total num: 105


backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}
  linear.{bias, weight}


Streaming output truncated to the last 5000 lines.
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[04/07 19:55:01 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[04/07 19:55:01 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.02 seconds.
[04/07 19:55:01 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[04/07 19:55:01 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.00 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.926
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.669
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.579
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.626
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.277
 Average Recall     (AR) @[ IoU=0.5

In [ ]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# MODEL.RPN.BBOX_REG_LOSS_TYPE
# MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE
# MODEL.RETINANET.BBOX_REG_LOSS_TYPE
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "giou"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "giou"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "giou"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/08 09:06:12 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

R-101.pkl: 179MB [00:08, 22.2MB/s]                           


[04/08 09:06:23 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[04/08 09:06:23 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up - Total num: 105


backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.deconv.{bias, weight}
roi_heads.mask_head.mask_fcn1.{bias, weight}
roi_heads.mask_head.mask_fcn2.{bias, weight}
roi_heads.mask_head.mask_fcn3.{bias, weight}
roi_heads.mask_head.mask_fcn4.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}
  fc1000.{bias, weight}


[04/08 09:06:23 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Streaming output truncated to the last 5000 lines.
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.190
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.492
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.507
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.514
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.435
[04/08 12:13:08 d2.evaluation.coco_evaluation]: Evaluation results for segm: 
|   AP   |  AP50  |  AP75  |  APs   |  APm   |  APl   |
|:------:|:------:|:------:|:------:|:------:|:------:|
| 43.549 | 83.659 | 43.767 | 29.120 | 44.569 | 41.034 |
[04/08 12:13:08 d2.engine.defaults]: Evaluation results for my_dataset_val in csv format:
[04/08 12:13:08 d2.evaluation.testing]: copypaste: Task: bbox
[04/08 12:13:08 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm

In [ ]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.975
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# MODEL.RPN.BBOX_REG_LOSS_TYPE
# MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE
# MODEL.RETINANET.BBOX_REG_LOSS_TYPE
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "ciou"
cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/13 16:38:32 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/13 16:40:17 detectron2]: Loss on Validation  done 11/97. 0.0001 s / img. ETA=0:00:36
[04/13 16:40:22 detectron2]: Loss on Validation  done 23/97. 0.0001 s / img. ETA=0:00:31
[04/13 16:40:27 detectron2]: Loss on Validation  done 35/97. 0.0001 s / img. ETA=0:00:26
[04/13 16:40:32 detectron2]: Loss on Validation  done 48/97. 0.0001 s / img. ETA=0:00:20
[04/13 16:40:38 detectron2]: Loss on Validation  done 60/97. 0.0001 s / img. ETA=0:00:15
[04/13 16:40:44 detectron2]: Loss on Validation  done 72/97. 0.0001 s / img. ETA=0:00:10
[04/13 16:40:49 detectron2]: Loss on Validation  done 84/97. 0.0001 s / img. ETA=0:00:05
[04/13 16:40:54 detectron2]: Loss on Validation  done 96/97. 0.0001 s / img. ETA=0:00:00


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/13 16:40:54 d2.utils.events]:  eta: 1:24:06  iter: 4219  total_loss: 0.3648  loss_cls: 0.1248  loss_box_reg: 0.04081  loss_mask: 0.1607  loss_rpn_cls: 0.007515  loss_rpn_loc: 0.0238  validation_loss: 0.4031    time: 3.3158  last_time: 3.0479  data_time: 1.8367  last_data_time: 1.1805   lr: 0.001  max_mem: 9263M
WARNING [04/13 16:41:56 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/13 16:41:56 d2.data.datasets.coco]: Loaded 97 images in COCO format from /content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json
[04/13 16:41:56 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/13 16:41:56 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/13 16:41:56 d2.data.common]: Serializing 97 elements to byte tensors and concatenating them

...

[04/13 18:18:32 detectron2]: Loss on Validation  done 11/97. 0.0001 s / img. ETA=0:00:09
[04/13 18:18:37 detectron2]: Loss on Validation  done 46/97. 0.0001 s / img. ETA=0:00:07
[04/13 18:18:43 detectron2]: Loss on Validation  done 71/97. 0.0001 s / img. ETA=0:00:04
[04/13 18:18:47 d2.utils.events]:  eta: 0:00:00  iter: 5999  total_loss: 0.3117  loss_cls: 0.1135  loss_box_reg: 0.04091  loss_mask: 0.1395  loss_rpn_cls: 0.005292  loss_rpn_loc: 0.02229  validation_loss: 0.3956    time: 2.2158  last_time: 2.2213  data_time: 0.0860  last_data_time: 0.0783   lr: 0.001  max_mem: 9264M
[04/13 18:18:47 d2.engine.hooks]: Overall training speed: 1798 iterations in 1:06:23 (2.2158 s / it)
[04/13 18:18:47 d2.engine.hooks]: Total training time: 1:39:35 (0:33:11 on hooks)
WARNING [04/13 18:18:47 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/13 18:18:47 d2.data.datasets.coco]: Loaded 97 images in COCO format from /content/drive/My

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/13 18:18:49 d2.evaluation.evaluator]: Inference done 11/97. Dataloading: 0.0019 s/iter. Inference: 0.1343 s/iter. Eval: 0.0096 s/iter. Total: 0.1459 s/iter. ETA=0:00:12
[04/13 18:18:54 d2.evaluation.evaluator]: Inference done 45/97. Dataloading: 0.0030 s/iter. Inference: 0.1347 s/iter. Eval: 0.0114 s/iter. Total: 0.1492 s/iter. ETA=0:00:07
[04/13 18:18:59 d2.evaluation.evaluator]: Inference done 78/97. Dataloading: 0.0033 s/iter. Inference: 0.1350 s/iter. Eval: 0.0124 s/iter. Total: 0.1509 s/iter. ETA=0:00:02


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/13 18:19:02 d2.evaluation.evaluator]: Total inference time: 0:00:13.724593 (0.149180 s / iter per device, on 1 devices)
[04/13 18:19:02 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:12 (0.134073 s / iter per device, on 1 devices)
[04/13 18:19:02 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[04/13 18:19:02 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[04/13 18:19:02 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[04/13 18:19:02 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.03 seconds.
[04/13 18:19:02 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[04/13 18:19:02 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.01 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.488
 Average Precision  (AP) @[ IoU=0.50      | are

In [11]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode

cfg = get_cfg()
cfg["MODEL"]['BACKBONE']

CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 2})

In [12]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# cfg["MODEL"]['BACKBONE'] = CfgNode({'NAME': 'build_resnet_backbone', 'FREEZE_AT': 0})
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.CHECKPOINT_PERIOD = 600
cfg.TEST.EVAL_PERIOD = 40
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 6000
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_MEAN = [26.9, 26.9, 26.9]
cfg.MODEL.PIXEL_STD = [34.4, 34.4, 34.4]
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# MODEL.RPN.BBOX_REG_LOSS_TYPE
# MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE
# MODEL.RETINANET.BBOX_REG_LOSS_TYPE
# cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
# cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "ciou"
# cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
cfg.OUTPUT_DIR = f"./Detectron2/Test_results/SGD0_{cfg.SOLVER.MOMENTUM }_{cfg.SOLVER.BASE_LR}_R101_{cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE}"

In [13]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[04/14 19:45:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/14 19:47:09 detectron2]: Loss on Validation  done 11/97. 0.0001 s / img. ETA=0:00:33
[04/14 19:47:15 detectron2]: Loss on Validation  done 24/97. 0.0001 s / img. ETA=0:00:30
[04/14 19:47:20 detectron2]: Loss on Validation  done 37/97. 0.0001 s / img. ETA=0:00:24
[04/14 19:47:25 detectron2]: Loss on Validation  done 49/97. 0.0001 s / img. ETA=0:00:19
[04/14 19:47:30 detectron2]: Loss on Validation  done 61/97. 0.0001 s / img. ETA=0:00:15
[04/14 19:47:35 detectron2]: Loss on Validation  done 72/97. 0.0001 s / img. ETA=0:00:10
[04/14 19:47:40 detectron2]: Loss on Validation  done 83/97. 0.0001 s / img. ETA=0:00:05
[04/14 19:47:46 detectron2]: Loss on Validation  done 95/97. 0.0001 s / img. ETA=0:00:00


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/14 19:47:47 d2.utils.events]:  eta: 1:54:50  iter: 3619  total_loss: 0.6075  loss_cls: 0.1212  loss_box_reg: 0.2923  loss_mask: 0.1554  loss_rpn_cls: 0.006932  loss_rpn_loc: 0.032  validation_loss: 0.7413    time: 2.8711  last_time: 3.1088  data_time: 1.2791  last_data_time: 0.9261   lr: 0.001  max_mem: 9262M
WARNING [04/14 19:48:50 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/14 19:48:50 d2.data.datasets.coco]: Loaded 97 images in COCO format from /content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json
[04/14 19:48:50 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/14 19:48:50 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/14 19:48:50 d2.data.common]: Serializing 97 elements to byte tensors and concatenating them a

...

[04/14 21:56:48 d2.utils.events]:  eta: 0:00:00  iter: 5999  total_loss: 0.5252  loss_cls: 0.09025  loss_box_reg: 0.2546  loss_mask: 0.1372  loss_rpn_cls: 0.003279  loss_rpn_loc: 0.0246  validation_loss: 0.7305    time: 2.1832  last_time: 2.1834  data_time: 0.0881  last_data_time: 0.0663   lr: 0.001  max_mem: 9264M
[04/14 21:56:48 d2.engine.hooks]: Overall training speed: 2398 iterations in 1:27:15 (2.1833 s / it)
[04/14 21:56:48 d2.engine.hooks]: Total training time: 2:10:35 (0:43:19 on hooks)
WARNING [04/14 21:56:48 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/14 21:56:48 d2.data.datasets.coco]: Loaded 97 images in COCO format from /content/drive/MyDrive/Thesis_Organized/Data/COCO_Format/coco_val_data.json
[04/14 21:56:48 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/14 21:56:48 d2.data.common

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/14 21:56:51 d2.evaluation.evaluator]: Inference done 11/97. Dataloading: 0.0056 s/iter. Inference: 0.1636 s/iter. Eval: 0.0091 s/iter. Total: 0.1783 s/iter. ETA=0:00:15
[04/14 21:56:56 d2.evaluation.evaluator]: Inference done 46/97. Dataloading: 0.0027 s/iter. Inference: 0.1377 s/iter. Eval: 0.0081 s/iter. Total: 0.1487 s/iter. ETA=0:00:07
[04/14 21:57:01 d2.evaluation.evaluator]: Inference done 82/97. Dataloading: 0.0024 s/iter. Inference: 0.1344 s/iter. Eval: 0.0079 s/iter. Total: 0.1448 s/iter. ETA=0:00:02


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/14 21:57:03 d2.evaluation.evaluator]: Total inference time: 0:00:13.535640 (0.147127 s / iter per device, on 1 devices)
[04/14 21:57:03 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:12 (0.134663 s / iter per device, on 1 devices)
[04/14 21:57:03 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[04/14 21:57:03 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[04/14 21:57:03 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[04/14 21:57:03 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.06 seconds.
[04/14 21:57:03 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[04/14 21:57:03 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.01 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.556
 Average Precision  (AP) @[ IoU=0.50      | are